In [1]:
import collections

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.metrics
import sklearn.preprocessing
import tensorflow as tf

from vae_bernouilli import VariationalMnist, train_vae_bernouilli, encoding

#sns.set(font_scale=1.5, palette='colorblind')
sns.reset_orig()

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
vae = VariationalMnist(28, 3)

In [4]:
scaler = sklearn.preprocessing.StandardScaler()

inverse_scale = lambda x, scaler: scaler.inverse_transform(x.reshape(-1, 28*28)).reshape(-1, 28, 28)

x_train_n = scaler.fit_transform(x_train.reshape(-1, 28*28)).reshape(-1, 28, 28)
x_test_n = scaler.transform(x_test.reshape(-1, 28*28)).reshape(-1, 28, 28)

/Users/srom/workspace/ds/env/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/srom/workspace/ds/env/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/srom/workspace/ds/env/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [5]:
save_path, best_elbo = train_vae_bernouilli(
    vae, x_train_n, x_test_n, 0.0001, 10000, 100, save_path='./vae_bernouilli.ckpt', best_elbo=532.865)

INFO:tensorflow:Restoring parameters from ./vae_bernouilli.ckpt
0 / 10000: latest = 534.1364041541162 | best = 534.1364041541162
1000 / 10000: latest = 307.92270836335905 | best = 534.1364041541162
2000 / 10000: latest = -1526.231314344567 | best = 534.1364041541162
3000 / 10000: latest = -1504.1705758529235 | best = 534.1364041541162
4000 / 10000: latest = -4932.884539762535 | best = 534.1364041541162
5000 / 10000: latest = -13459.881697788984 | best = 534.1364041541162
6000 / 10000: latest = -14056.31020361178 | best = 534.1364041541162
7000 / 10000: latest = -15022.571205475098 | best = 534.1364041541162


KeyboardInterrupt: 

In [ ]:
x_encoded = encoding(vae, x_test_n[:300], './vae_bernouilli.ckpt')

In [ ]:
def plot_encoding(X_code, y, figsize=(15, 8)):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')
    
    vals = np.unique(y)
    
    data_per_class = collections.defaultdict(list)
    for i, v in enumerate(y):
        x_c = X_code[i]
        data_per_class[v].append(x_c)
    
    for v in sorted(data_per_class.keys()):
        data = data_per_class[v]
        x1 = [i for i, _, _ in data]
        x2 = [j for _, j, _ in data]
        x3 = [k for _, _, k in data]
        ax.scatter(x1, x2, x3, label=v)
        
    ax.legend()

In [ ]:
plot_encoding(x_encoded, y_test[:300])

In [ ]:
def generate(vae, num_samples, save_path):
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, save_path)
        return sess.run(vae.samples, {vae.num_samples: num_samples})

In [ ]:
digits_n = generate(vae, 10, save_path)

In [ ]:
digits = inverse_scale(digits_n, scaler).reshape((-1, 28, 28))

In [ ]:
plt.imshow(digits[1], cmap='Greys_r')
plt.grid(False)

In [ ]:
best_elbo